In [1]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy import create_engine

In [2]:
# 1.

driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'  

Your password: ········


In [3]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [4]:
db_connection = sa.create_engine(connection_string)

In [5]:
# 2.

def rentals_month(month,year,engine):
    
    query = f'SELECT * FROM sakila.rental WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};'
    df = pd.read_sql_query(query, engine)
    
    return df

In [38]:
df = rentals_month(5,2005,db_connection)

In [39]:
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [16]:
# 3.

def rental_count_month(df):
    
    year = str(df['rental_date'].dt.year[0])
    month = str(df['rental_date'].dt.month[0])
    
    df_customer = df.groupby(df['customer_id'])[['rental_id']].count()
    df_customer = df_customer.rename(columns={"rental_id":f"rentals_{month}_{year}"})
    
    return df_customer

In [76]:
# 4.

def compare_rentals(df1,df2):
    
    df = df1.join(df2,on='customer_id')
    df = df.fillna(0)
    df['diffrence'] = df.iloc[:,0] - df.iloc[:,1]
    
    return df

In [63]:
df_5 = rentals_month(5,2005,db_connection)

In [66]:
df_5 = rental_count_month(df_5)

In [64]:
df_6 = rentals_month(6,2005,db_connection)

In [67]:
df_6 = rental_count_month(df_6)

In [78]:
df = compare_rentals(df_5,df_6)

In [79]:
df

,rentals_5_2005,rentals_6_2005,diffrence
customer_id,,,
1,2,7.0,-5.0
2,1,1.0,0.0
3,2,4.0,-2.0
5,3,5.0,-2.0
6,3,4.0,-1.0
...,...,...,...
594,4,6.0,-2.0
595,1,2.0,-1.0
596,6,2.0,4.0
